In [74]:
import pandas as pd
import scanpy as sc
import anndata as ad
import numpy as np
import seaborn as sns
from tqdm import tqdm
import os

In [75]:
def filter_df(df):
    return df[(df['pvals_adj'] <= 0.001) & (df['scores'] >= 0)]

def find_files(directory, substring, extension):
    # 存储符合条件的文件路径
    matching_files = []
    # 遍历给定目录及其所有子目录
    for root, dirs, files in os.walk(directory):
        # 检查每个文件
        for file in files:
            # 检查文件名是否包含特定子字符串并且以特定扩展名结尾
            if substring in file and file.endswith(extension):
                # 如果符合条件，添加完整路径到列表
                matching_files.append(os.path.join(root, file))
    return matching_files

In [76]:
files = find_files('/data/twang15/spatial_protein/CODEX_data/hubmap/integrated/lymph_node', 'integrated', '.h5ad')


In [71]:
files[0].split('/')[-1].split('_')[0] + '_' + files[0].split('/')[-1].split('_')[1]

'HBM673.TNZM.339_reg001'

In [77]:
for file in tqdm(files):
    tissue = file.split('/')[-2]
    id = file.split('/')[-1].split('_')[0] + '_' + file.split('/')[-1].split('_')[1]
    adata = sc.read_h5ad(file)
    adata = adata[adata.obs['cell_type'].astype(str).map(adata.obs['cell_type'].value_counts()) >= 10]
    adata = adata[:, ~adata.var_names.isna()]
    adata.var["mt"] = adata.var_names.str.startswith("MT-")
    adata = adata[:, ~adata.var_names.str.startswith("MT-")]
    sc.pp.filter_cells(adata, min_genes=200)
    sc.pp.filter_genes(adata, min_cells=30)
    sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)
    sc.tl.rank_genes_groups(adata, "cell_type", method="t-test")
    rec_arrays = ['names', 'scores', 'pvals', 'pvals_adj', 'logfoldchanges']

    # 构建字典，按 dtype_name 将对应的各数组合并为 DataFrame，并添加 'differential group' 列
    dfs = {
        dtype_name: pd.DataFrame({
            rec: adata.uns['rank_genes_groups'][rec][dtype_name]
            for rec in rec_arrays
        }).assign(**{'Cell Type': dtype_name})  # 添加一列 'differential group' 来标识 dtype_name
        for dtype_name in adata.uns['rank_genes_groups']['names'].dtype.names
    }
    filtered_dfs = pd.concat([filter_df(df) for df in dfs.values()])
    os.makedirs(f'/data/twang15/spatial_protein/CODEX_data/hubmap/differential_gene/{tissue}/{id}/', exist_ok=True)
    filtered_dfs.to_csv(f'/data/twang15/spatial_protein/CODEX_data/hubmap/differential_gene/{tissue}/{id}/cell_type_differential.tsv', sep='\t')
    
    sc.tl.rank_genes_groups(adata, "neighborhood10", method="t-test")
    rec_arrays = ['names', 'scores', 'pvals', 'pvals_adj', 'logfoldchanges']

    # 构建字典，按 dtype_name 将对应的各数组合并为 DataFrame，并添加 'differential group' 列
    dfs = {
        dtype_name: pd.DataFrame({
            rec: adata.uns['rank_genes_groups'][rec][dtype_name]
            for rec in rec_arrays
        }).assign(**{'Neighborhood': dtype_name})  # 添加一列 'differential group' 来标识 dtype_name
        for dtype_name in adata.uns['rank_genes_groups']['names'].dtype.names
    }
    filtered_dfs = pd.concat([filter_df(df) for df in dfs.values()])
    filtered_dfs.to_csv(f'/data/twang15/spatial_protein/CODEX_data/hubmap/differential_gene/{tissue}/{id}/neighborhood_differential.tsv', sep='\t')
    

  0%|          | 0/23 [00:00<?, ?it/s]

/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/tmp/ipykernel_2086885/2749894378.py:7: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var["mt"] = adata.var_names.str.startswith("MT-")
/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/scanpy/preprocessing/_simple.py:139: ImplicitModificationWarning: Trying

/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:419: RuntimeWarning: overflow encountered in expm1
  foldchanges = (self.expm1_func(mean_group) + 1e-9) / (
/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:419: RuntimeWarning: overflow encountered in expm1
  foldchanges = (self.expm1_func(mean_group) + 1e-9) / (
/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:419: RuntimeWarning: overflow encountered in expm1
  foldchanges = (self.expm1_func(mean_group) + 1e-9) / (
/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:396: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, '

  4%|▍         | 1/23 [29:56<10:58:51, 1796.91s/it]/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/tmp/ipykernel_2086885/2749894378.py:7: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var["mt"] = adata.var_names.str.startswith("MT-")
/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/scanpy/preprocessing/

/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:419: RuntimeWarning: overflow encountered in expm1
  foldchanges = (self.expm1_func(mean_group) + 1e-9) / (
/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:419: RuntimeWarning: overflow encountered in expm1
  foldchanges = (self.expm1_func(mean_group) + 1e-9) / (
/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:396: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, 'names'] = self.var_names[global_indices]
/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:398: PerformanceWarning: DataFrame is highly fragmented.  This is u

...

 91%|█████████▏| 21/23 [2:29:39<06:39, 199.96s/it]/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/tmp/ipykernel_2086885/2749894378.py:7: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var["mt"] = adata.var_names.str.startswith("MT-")
/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/scanpy/preprocessing/_

/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:419: RuntimeWarning: overflow encountered in expm1
  foldchanges = (self.expm1_func(mean_group) + 1e-9) / (
/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:419: RuntimeWarning: overflow encountered in expm1
  foldchanges = (self.expm1_func(mean_group) + 1e-9) / (
/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:396: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, 'names'] = self.var_names[global_indices]
/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:398: PerformanceWarning: DataFrame is highly fragmented.  This is u

 96%|█████████▌| 22/23 [2:30:14<02:30, 150.60s/it]/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/tmp/ipykernel_2086885/2749894378.py:7: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var["mt"] = adata.var_names.str.startswith("MT-")
/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/scanpy/preprocessing/_

/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:419: RuntimeWarning: overflow encountered in expm1
  foldchanges = (self.expm1_func(mean_group) + 1e-9) / (
/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:419: RuntimeWarning: overflow encountered in expm1
  foldchanges = (self.expm1_func(mean_group) + 1e-9) / (
/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:396: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, 'names'] = self.var_names[global_indices]
/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:398: PerformanceWarning: DataFrame is highly fragmented.  This is u

100%|██████████| 23/23 [2:31:39<00:00, 395.65s/it]


In [73]:
adata = sc.read_h5ad('/data/twang15/spatial_protein/CODEX_data/hubmap/integrated/large_intestine/HBM245.NHMB.685_reg001_integrated_protein.h5ad')
adata = adata[adata.obs['cell_type'].map(adata.obs['cell_type'].value_counts()) >= 10]
adata = adata[:, ~adata.var_names.isna()]
adata.var["mt"] = adata.var_names.str.startswith("MT-")
adata = adata[:, ~adata.var_names.str.startswith("MT-")]
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=30)
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)

/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/tmp/ipykernel_2086885/1125219341.py:4: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var["mt"] = adata.var_names.str.startswith("MT-")
/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/scanpy/preprocessing/_simple.py:139: ImplicitModificationWarning: Trying

In [42]:
sc.tl.rank_genes_groups(adata, "cell_type", method="t-test")
rec_arrays = ['names', 'scores', 'pvals', 'pvals_adj', 'logfoldchanges']

# 构建字典，按 dtype_name 将对应的各数组合并为 DataFrame，并添加 'differential group' 列
dfs = {
    dtype_name: pd.DataFrame({
        rec: adata.uns['rank_genes_groups'][rec][dtype_name]
        for rec in rec_arrays
    }).assign(**{'Cell Type': dtype_name})  # 添加一列 'differential group' 来标识 dtype_name
    for dtype_name in adata.uns['rank_genes_groups']['names'].dtype.names
}
filtered_dfs = pd.concat([filter_df(df) for df in dfs.values()])
filtered_dfs.to_csv('')

/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:420: RuntimeWarning: overflow encountered in expm1
  self.expm1_func(mean_rest) + 1e-9
/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:422: RuntimeWarning: divide by zero encountered in log2
  self.stats[group_name, 'logfoldchanges'] = np.log2(
/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:420: RuntimeWarning: overflow encountered in expm1
  self.expm1_func(mean_rest) + 1e-9
/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:422: RuntimeWarning: divide by zero encountered in log2
  self.stats[group_name, 'logfoldchanges'] = np.log2(
/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:420: RuntimeWarning: overflow encountered in expm1
  self.expm1_func(mean_rest) + 1e-9
/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/sc

In [29]:
sc.tl.rank_genes_groups(adata, "neighborhood10", method="t-test")
rec_arrays = ['names', 'scores', 'pvals', 'pvals_adj', 'logfoldchanges']

# 构建字典，按 dtype_name 将对应的各数组合并为 DataFrame，并添加 'differential group' 列
dfs = {
    dtype_name: pd.DataFrame({
        rec: adata.uns['rank_genes_groups'][rec][dtype_name]
        for rec in rec_arrays
    }).assign(**{'Neighborhood': dtype_name})  # 添加一列 'differential group' 来标识 dtype_name
    for dtype_name in adata.uns['rank_genes_groups']['names'].dtype.names
}



/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:419: RuntimeWarning: overflow encountered in expm1
  foldchanges = (self.expm1_func(mean_group) + 1e-9) / (
/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:420: RuntimeWarning: overflow encountered in expm1
  self.expm1_func(mean_rest) + 1e-9
/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:419: RuntimeWarning: invalid value encountered in true_divide
  foldchanges = (self.expm1_func(mean_group) + 1e-9) / (
/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:419: RuntimeWarning: overflow encountered in expm1
  foldchanges = (self.expm1_func(mean_group) + 1e-9) / (
/data/twang15/mamba/envs/mip/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:420: RuntimeWarning: overflow encountered in expm1
  self.expm1_func(mean_rest) + 1e-9
/data/twang15/mamba/envs/mip/li